In [1]:
import numpy as np

In [2]:
def output(out):
    if type(out)==str:
        print(out)
    elif type(out)==list:
        st=''
        for i in out:
            st+=str(i)+' '
        print(st)

# Problem 1: Finding a Motif in DNA

#### Function (Brut force)

In [3]:
def motif_finder(string,sub):
    num=[]
    for i in range(len(seq)):
        if seq[i]==sub[0]:
            flag=1
            if i+len(sub)<=len(seq):
                for j in range(i,i+len(sub)):
                    if seq[j]==sub[j-i]:
                        flag+=1
            if flag==len(sub)+1:
                num.append(i+1)
    return num

#### Application

In [4]:
seq=''
with open("sequence (3).fasta",'r') as f:
    for i in f:
        s=f.readline()
        seq+=str(s[:-1])
sub='TTAAAACCGCTACAACAGGTGAGCGGTCCGTTAGGTGGTCTGATCTCGAGATGGAAATGGTGGCACGTGTTGCGCTGGTTCAGCCACACGAGCCAGGAGCGACGACCGTGGTGAACGGATGCTGGTACGCTCCGGGCGTAGCCGTTTTTCGCTGTCTACCCTGCCAGCGGCGGATTTCCAGCGACCCAGTTTTCGATGGCGCATCAGGACGTTCGCTATTACTTAAATGGTATGCTGTTTGAAACCGAACTTTGCCAAGCCATTCGGTGATCGTACCGCGTAAAGGCGTGATTGAACTGATGCGTATGCTCGACGGCGGTCCAAACTGGTGGATGGTCGCTTCCCGGATTACCGCCGCGTTCTGCCGAAGAATCCGGACAAACATCTGGCGTGCGCCTGTATGTCAGCGAAAACCAGCTGAAAATCACCGCCAACAACCCGGAACAGGAAGAAGCGGAATTCTGAACGCGCTGAAATGCGAAAACGTCCGCATGATGCTGACCGATTCGGTTTCCAGCGTGCAGATTGATGATCCGCGATTTCCGCAACATTGAAACCGCGGATCTCGCTTTATCTCCCGGCTTTAACTTTCTGGTAGGAGTTTGCAGATTGGTCGCGTCATTCGCCATGAGCAGGAGGCATTTGTTCTCCATGGGCGATTACAGGGCGTGACGGGCATAAAGTCGCGGAACTGGCGCACCTGATGCCAATGCAGCTGATAACGCCAGAAGGGTTTACTTTTTCACCGCCTGGAGCAATCTCAAGCGATTGCTCAAGCAGCGCAATGCGGCGCTGCGCCAGGTGACACGGAGTATAGCGCCGGTATCGCGGCCGATATGGCCGATACCTGTAAGCAATTTCTCCCTGAGTTTTCTCTGACGATCGCCAGCTAACCTACACCGCGCATGGCCCGCATAAAGCGGACTTACGCATTCGCGCCGACGGTGCGAGTTCCTCACCCGTGAAAGCGGGCGGCGGTGTCTCTACCTGATAGATGATTTTGCCTCTGAGCTTGATGAGCTGAACACGTTATAGACATGTCGGACGAAAATTCGAAGATGTTTACCGTGGAAAAGGGTAAAATAACGGTGAAAGGGCTGGATGCGGTGCGTAAGCGCCCGGGTATGTATATCGGCGACACGGATGACGGCACCGGTCTATCGTCACCATTCACGCCGACAACTCTGTCTCTGTACAGGATGACGGGCGCGGCATTCCGACCGGTATTCTAACTCCTATAAAGTGTCCGGCGGTCTGCACGGC'

In [5]:
%%time
res=motif_finder(seq,sub)
output(res)

522 
CPU times: user 1min 52s, sys: 155 ms, total: 1min 52s
Wall time: 1min 52s


#### Knuth–Morris–Pratt algorithm

In [6]:
def z_func(s):
    s+='$'
    l,r= 0,0
    z=[0] * len(s)
    for i in range(1,len(s)):
        z[i]=max(0, min(z[i-l],r-i))
        while s[z[i]]==s[i+z[i]]:
            z[i]+=1
        if i+z[i]>r:
            l,r=i,i+z[i]
    return z[:-1]

In [7]:
def KMP(s,t):
    k=0
    a=[]
    i=0
    while i<(len(t)):
        while k>0 and t[i]!=s[k]:
            k=z_func(s[:k+1])[0]
        if t[i]==s[k]:
            k+=1
        if k==len(s):
            a.append(i-len(s)+2)
            k=z_func(s[:k+1])[0]
            i-=(len(s)-1)
        i+=1
    return a

In [8]:
%%time
res=KMP(sub,seq)
output(res)

522 
CPU times: user 2.1 s, sys: 3.04 ms, total: 2.1 s
Wall time: 2.1 s


KMP algorithm has been chosen due to its' simplicity in programming and as it is acceptably fast in comparisson with brut force one. 

# Problem 2: Genome Assembly with Perfect Coverage

In [9]:
def wrap_around(genome):
    identical=[]
    linear=genome
    for i in range(len(linear)):
        linear=linear[1:]+linear[0]
        identical.append(linear)
    return identical

In [10]:
wrap_around('ACGTAC')

['CGTACA', 'GTACAC', 'TACACG', 'ACACGT', 'CACGTA', 'ACGTAC']

In [11]:
reads=[]
with open("rosalind_pvoc.txt",'r') as f:
    for i in f:
        s=str(i)
        reads.append(str(s[:-1]))
print(len(reads))
print(len(reads[0]))

2000
50


In [12]:
def SymbolToNumber(sym):
    if sym=='A':
        return 0
    elif sym=="C":
        return 1
    elif sym=="G":
        return 2
    elif sym=="T":
        return 3
    else:
        return 0

In [13]:
def PatternToNumber(text):
    if text=="":
        return 0
    sym=text[-1]
    prefix=text[:len(text)-1]
    res=PatternToNumber(prefix)*4+SymbolToNumber(sym)
    return res

In [14]:
def prefix_suffix(reads):
    pre_suf=[]
    for i in reads:
        pre_suf.append((i[:-1],i[1:]))
    return pre_suf

In [16]:
def graf_cons(reads):
    pre_suf=prefix_suffix(reads)
    edges=[]
    seq=reads
    graf=[]
    nodes=[]
    for i in range(len(pre_suf)):
        k=0
        for j in range(i,len(pre_suf)):
            if pre_suf[i]!=pre_suf[j]:
                k+=1
        if k==len(pre_suf)-i-1:
            nodes.append(pre_suf[i])
            edges.append(seq[i])
    for i in range(len(nodes)):
        graf.append((nodes[i][0],edges[i]))
    return nodes,edges

In [17]:
def adj_row(reads):
    n=len(reads[0])-1
    row=[]
    nodes,_=graf_cons(reads)
    for i in range(len(nodes)):
        row.append((PatternToNumber(nodes[i][0]),PatternToNumber(nodes[i][1]),nodes[i][0],nodes[i][1]))
    return row
            

In [18]:
def finding(reads):
    g=adj_row(reads)
    final=''
    final+=g[0][2][-1]
    l=0
    start=g[0][0]
    for i in range(len(g)):
        for k in range(len(g)):
            if g[l][1]==g[k][0]:
                final+=g[k][2][-1]
                l=k
                break
        if g[l][0]==start:
            final=final[:-1]
            break
    return final

In [19]:
ref='CGTGCGCACTCTAAAGCCCCATCGCTTATAGCTTAGACCGTGGTTAGGTTTTCCTACAGTTGGATGGGTCCAACGTCTTATCCCTCCAATGCAAGTATCCGAACTAACAGGCGAATAGGATTTCCCGGAGGGCTTAGGACGATGCCTCATCGGTTACCAGCGGGTCCAGATGTAACGTCGCTCCACAGAACATGCTCAGGACCGACTGCTTAGAATCGCCTAAATATACGTCTCGCCGGTCCCCTGACATATGCTGTAATAAGTTAACTAAGCTCGCAGGTGTAATATTTAACAGAGTCTAAAACCGCGAAGCATTACCACGCATGTCGCTAGGGTAGATTTACACCATTTGTCCCTTGCACCCGTTGGTGAACGAGGCCGGGGGTATAATGTCGTGGGTTTACCCATGAATAAGTAATGGACTGGCAGACCTTGCACTATTTGTAATCCTCGCCGCGGCCAAACAACCCGCGACCGGTCATGTACGCCGTATGCTTGGTTAAGCTCGCTTCAGAACAACCATCCGCGTTTTATCTAGTGCGCTCGTAGTCTTGGATGGCGTTCAACGATAGGATGTCCGCGCTGTGTTAGTTCGCTCCCAATATGTATTAGCAACGGTCTGCCATCAGGCCCTAATAATGCCTGGAGCATAGCGGTAATTCACGTCGAACACCTCCCATATTCACTTGGTGTACCAACTCATTTGTTAGATTAACGTTCTTTAGTACCATCTACGCGGTTATTTCACCCTGGGCTACTGCTGCAAATCCAAAGACCGGCTCCTAAACGGCCAGTTTGATTCCTCGCCTGACAAAGTCCTTCCCGCGGCTCCCAACACGAAGAACATATCATGATGTTGAGCTCAAATTCTATACGTGCTGCTAGTATCCGTGTACTCTATCGAGTTGGTTAGGCGTGCCGTACCACAGAATAGAGACATCAGCGGCAAGCAGTGCGAATTATAAGCTCCAGCGAAACACGAAGCTACATGTGCAAACCAAGTAGCCATAGTTCCGACATCCGGTGCTACCACCCAATCCTTTTAGAGCGATGAGACGAAATACACTTTCGTGTGCCCTCGTGGGATTATCGATAACCATAGTGTCCGATGTATACGTATTAAGAGGACGGGCAAACACGTGTGGAGCGGTCTGGAGAGGGAATGCATAACGTCGAGATTCCCAGCGCTCGCTTGATCTGAAACACGAGGGGATGGGTTCGTACTAACCTTGTGCGGCTATTGCCTATGTATCTTTGCGTGAACACTGACACTCGTAACTCCGGAATCACATTATCCGTACTTCTGGAACGGTTGTTGGTCTAGTTCGAGGCGACTGAACTTGCGTTAGTTAGGCAGCAACACCCCCAGGCAGTCAGGACCCGGGGTTCCATGGCTCCTCTCCCTAGAGATTTCGATAGTAAGTATCTTGCCCATGGAGTATCGACTCACCGCTATTGCCACGCCTGATGTACTTGTGGACATAACGCTTTTAAGACAGCTTGTGGTTTCAGTACATTTCGGGTCCAGATTTACCAGGATCATTACTCGATTCGCGGTTTAACCGGTCTCTTCACCCGCTATCCACTGAGGGATCGCTATAAAAGAGACTGCGCAGGATGTACTTTTCGCAATACGTAACTACTTCGGATTGTATCCCTACGAATCCGATCAGACCTCGGCCTTTACACTGCGCGATCCCTGGTGTTATCATACAGTATACGCACCCACACTCATTTGTAGCGCCATCGGTTTCATGAAACCTGTGTAAGGAACGGAGTGGAGAAGCCCCTTATACTTTGTCAGTTGAGTCGCCTGACGCACGAAAGGTCAAAATTCTGCATGAGTCGGAGCGTGGAGTCAGCAGCGAATGGGAGCGGACACCATAGACTACCGACTCGCATACCAATATCCATAGCCAAGACTGCAGCGTACTCCGGTGAAAGCGGTTTGGCTCTAGATGATATTAACAAAGATAAGGTAGTCACTCGATTGGGACCAT'
print(len(ref))

2000


In [20]:
%%time
final=finding(reads)
a=wrap_around(ref)
for i in a:
    if i==final:
        print(final)

CACCCCCAGGCAGTCAGGACCCGGGGTTCCATGGCTCCTCTCCCTAGAGATTTCGATAGTAAGTATCTTGCCCATGGAGTATCGACTCACCGCTATTGCCACGCCTGATGTACTTGTGGACATAACGCTTTTAAGACAGCTTGTGGTTTCAGTACATTTCGGGTCCAGATTTACCAGGATCATTACTCGATTCGCGGTTTAACCGGTCTCTTCACCCGCTATCCACTGAGGGATCGCTATAAAAGAGACTGCGCAGGATGTACTTTTCGCAATACGTAACTACTTCGGATTGTATCCCTACGAATCCGATCAGACCTCGGCCTTTACACTGCGCGATCCCTGGTGTTATCATACAGTATACGCACCCACACTCATTTGTAGCGCCATCGGTTTCATGAAACCTGTGTAAGGAACGGAGTGGAGAAGCCCCTTATACTTTGTCAGTTGAGTCGCCTGACGCACGAAAGGTCAAAATTCTGCATGAGTCGGAGCGTGGAGTCAGCAGCGAATGGGAGCGGACACCATAGACTACCGACTCGCATACCAATATCCATAGCCAAGACTGCAGCGTACTCCGGTGAAAGCGGTTTGGCTCTAGATGATATTAACAAAGATAAGGTAGTCACTCGATTGGGACCATCGTGCGCACTCTAAAGCCCCATCGCTTATAGCTTAGACCGTGGTTAGGTTTTCCTACAGTTGGATGGGTCCAACGTCTTATCCCTCCAATGCAAGTATCCGAACTAACAGGCGAATAGGATTTCCCGGAGGGCTTAGGACGATGCCTCATCGGTTACCAGCGGGTCCAGATGTAACGTCGCTCCACAGAACATGCTCAGGACCGACTGCTTAGAATCGCCTAAATATACGTCTCGCCGGTCCCCTGACATATGCTGTAATAAGTTAACTAAGCTCGCAGGTGTAATATTTAACAGAGTCTAAAACCGCGAAGCATTACCACGCATGTCGCTAGGGTAGATTTACACCATTTGTCCCTTGC

# Problem 3: Bellman-Ford Algorithm

In [21]:
def reading_graphs(file):
    vert=[]
    edges=[]
    with open(file,'r') as f:
        f.readline()
 
        for i in f:
            s=str(i)
            s=s.split(' ')
            flag=0
            for j in range(len(vert)):
                if int(s[0])!=vert[j]:
                    flag+=1
            if s[2][-1]=='\n':
                s[2]=s[2][:-1]
            if flag==len(vert):
                vert.append(int(s[0]))
            edges.append(s)
    return vert, edges


In [22]:
def Bellman_Ford(vert, edges,source):
    dist=[]
    predec=[]
    for v in vert:
        dist.append(float('inf'))
        predec.append(None)
    dist[source-1]=0
    for i in range(1,len(vert)):
        for j in edges:
            if dist[int(j[0])-1]+int(j[2])<dist[int(j[1])-1]:
                dist[int(j[1])-1]=dist[int(j[0])-1]+int(j[2])
                predec[int(j[1])-1]=int(j[0])-1
    
    for i in edges:
        if dist[int(i[0])-1]+int(i[2])<dist[int(i[1])-1]:
            print ("Graph contains a negative-weight cycle")
            raise 
    for i in range(len(dist)):
        if dist[i]==float('inf'):
            dist[i]='x'
    return dist

In [23]:
%%time
source=1
vert, edges=reading_graphs("1.txt")
dist=Bellman_Ford(vert,edges,source)
output(dist)

0 5 5 6 9 7 9 8 x 
CPU times: user 833 µs, sys: 829 µs, total: 1.66 ms
Wall time: 903 µs


# Problem 4: Connected Components

In [24]:
def reading_graphs(file):
    vert=[]
    edges=[]
    with open(file,'r') as f:
        num=f.readline().split(' ')
        for i in f:
            s=str(i)
            s=s.split(' ')
            if s[1][-1]=='\n':
                s[1]=s[1][:-1]
            edges.append(s)
            
    for i in range(1,int(num[0])+1):
        vert.append(i)
    return vert, edges

In [25]:
global visited
global ccnum
vert,edges=reading_graphs('2.txt')
visited=[False]*(len(vert)+1)
ccnum=[0]*(len(vert)+1)
cc=0

In [26]:
def explore(vert,edges,v):
    visited[v]=True
    previsit(v)
    for i in edges:
        if int(i[0])!=v:
            continue
        else:
            if not visited[int(i[1])]:
                explore(vert,edges,int(i[1]))
    #postvisit(v)

In [27]:
def DFS(vert,edges):
    global cc
    for i in vert:
        if not visited[i]:
            cc+=1
            explore(vert,edges,i)
    print(max(ccnum))

In [28]:
def previsit(v):
    ccnum[v]=cc

In [29]:
%%time
DFS(vert,edges)

3
CPU times: user 183 µs, sys: 50 µs, total: 233 µs
Wall time: 210 µs


# Problem 5: Quick Sort

In [30]:
def input(file):
    with open(file,'r') as f:
        num=int(f.readline())
        array=[]
        s=f.readline()
        s=s.split(" ")
        for i in s:
            array.append(int(i))
    return array,0,num-1

In [31]:
global A
def quick_sort(A, low,high):
    if low<high:
        p=partition(A,low,high)
        quick_sort(A,low,p-1)
        quick_sort(A,p+1,high)


In [32]:
def partition(A,low,high):
    pivot=A[high]
    i=low
    for j in range(low,high):
        if A[j]<= pivot:
            A[i],A[j]=A[j],A[i]
            i+=1
    A[i],A[high]=A[high],A[i]
    return i

In [33]:
%%time
A, low, high=input('3.txt')
quick_sort(A, low,high)
output(A)

-10 -2 4 5 7 8 11 
CPU times: user 300 µs, sys: 293 µs, total: 593 µs
Wall time: 395 µs
